In [12]:
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re, string
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
fname = 'tweets.txt'

In [8]:
lines = [line.rstrip() for line in open(fname)]
file = open("newfile.txt", "w")

In [9]:
def checkTopic (topic, text):
    if (set(topic) & set(text)):
        return True
    return False

def bag_of_words(words):
    if len(words) > 0:
        return dict([(word, True) for word in words])

stop = stopwords.words('english')

In [10]:
json_input = lines[4]
decoded = json.loads(json_input)
    
# pretty printing of json-formatted string
print json.dumps(decoded, sort_keys=True, indent=4)

nf_timestamp = decoded['timestamp_ms']
print "nf_timestamp", nf_timestamp

nf_user = decoded['user']['screen_name']
print "nf_user", nf_user

{
    "contributors": null, 
    "coordinates": null, 
    "created_at": "Sun Jun 12 18:07:18 +0000 2016", 
    "entities": {
        "hashtags": [], 
        "symbols": [], 
        "urls": [], 
        "user_mentions": []
    }, 
    "favorite_count": 0, 
    "favorited": false, 
    "filter_level": "low", 
    "geo": null, 
    "id": 742055693281288200, 
    "id_str": "742055693281288193", 
    "in_reply_to_screen_name": null, 
    "in_reply_to_status_id": null, 
    "in_reply_to_status_id_str": null, 
    "in_reply_to_user_id": null, 
    "in_reply_to_user_id_str": null, 
    "is_quote_status": false, 
    "lang": "en", 
    "place": null, 
    "retweet_count": 0, 
    "retweeted": false, 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "text": "Thread of my fav mutuals \ud83d\udc96", 
    "timestamp_ms": "1465754838663", 
    "truncated": false, 
    "user": {
        "contributors_enabled": false, 
        "created_at":

In [11]:
total = 0
exception = 0
topic = ["terrorism"]
print topic

for index in range(len(lines)):
    json_input = lines[index]
    decoded = json.loads(json_input)
    
    try:

        nf_lang = decoded['user']['lang']
        #print "nf_lang", nf_lang
    
        if nf_lang == 'en':
            #print "JSON parsing example: ", decoded['hashtags']
            # cal treure entities and stop words
            nf_tweet_text = decoded['text']
            #print "nf_tweet_text", nf_tweet_text
            nf_RT = (decoded['retweet_count']) > 0
            #print "nf_RT", nf_RT
            if nf_RT:
                nf_rt_text = decoded['retweeted_status']['text']
                nf_tweet_text = nf_rt_text
            #print "nf_tweet_text :: ", nf_tweet_text

            #exclude punctuation in words
            regex = re.compile('[%s]' % re.escape(string.punctuation))
            nf_tweet_text = regex.sub('', nf_tweet_text)
            
            s = nf_tweet_text.split()          
            #print(s)
 
            # filtered = [word for words in s if word not in stopwords.words('english')]
            nf_tweet_filtered_text = [i for i in s if i not in stop]
            #print "nf_tweet_filtered_text :: ", nf_tweet_filtered_text
            nf_tweet_filtered_text = [element.lower() for element in nf_tweet_filtered_text]
            
            bag = bag_of_words(nf_tweet_filtered_text)

            # check the topic
            nf_positiu = checkTopic(topic, nf_tweet_filtered_text)
 
            nf_words = []
        
            nf_count_hashtags = len(decoded['entities']['hashtags'])
            #print "nf_count_hastags", nf_count_hashtags
            nf_pos_first_hashtag = 0
            nf_tweet_hashtags = []
            if nf_count_hashtags > 0:
                nf_pos_first_hashtag = decoded['entities']['hashtags'][0]['indices'][0]
                #print "nf_pos_first_hashtag", nf_pos_first_hashtag
                for i in range(nf_count_hashtags):
                    nf_tweet_hashtags.append(decoded['entities']['hashtags'][i]['text'])
                #print "nf_tweet_hashtags", nf_tweet_hashtags
                nf_tweet_hashtags = [element.lower() for element in nf_tweet_hashtags]
                nf_words.append(nf_tweet_hashtags)
        
            nf_count_urls = len(decoded['entities']['urls'])
            #print "nf_count_urls", nf_count_urls
            nf_pos_first_url = 0
            nf_tweet_urls = []
            if nf_count_urls > 0:
                nf_pos_first_url = decoded['entities']['urls'][0]['indices'][0]
                #print "nf_pos_first_url", nf_pos_first_url
                for i in range (nf_count_urls):
                    nf_tweet_urls.append(decoded['entities']['urls'][i]['url'])
                #print nf_tweet_urls
                #nf_tweet_urls = [element.lower() for element in nf_tweet_urls]
                #nf_words.append(nf_tweet_urls)
        
            nf_count_mentions = len(decoded['entities']['user_mentions'])
            #print "nf_count_mentions", nf_count_mentions
            nf_pos_first_mention = 0
            nf_tweet_mentions = []
            if nf_count_mentions > 0:
                nf_pos_first_mention = decoded['entities']['user_mentions'][0]['indices'][0]
                #print "nf_pos_first_mention", nf_pos_first_mention    
                for i in range(nf_count_mentions):
                    nf_tweet_mentions.append(decoded['entities']['user_mentions'][i]['screen_name']) 
                #print "nf_tweet_mentions", nf_tweet_mentions
                nf_tweet_mentions = [element.lower() for element in nf_tweet_mentions]
                nf_words.append(nf_tweet_mentions) 
            
            #print "nf_words", nf_words
            #print "topic", topic
            #print "nf_positiu", nf_positiu
            if nf_positiu:
                total = total + 1

            num_list = [0]
            if nf_pos_first_hashtag > 0:
                num_list.append(nf_pos_first_hashtag)
            if nf_pos_first_url > 0:
                num_list.append(nf_pos_first_url)
            if nf_pos_first_mention > 0:
                num_list.append(nf_pos_first_mention)
            nf_pos_first_entity = min(num_list)
            #print "nf_pos_first_entity", nf_pos_first_entity

            nf_user = decoded['user']['screen_name']
            #print "nf_user", nf_user

            nf_timestamp = decoded['timestamp_ms']
            #print "nf_timestamp", nf_timestamp
            
            #file.write(nf_tweet_filtered_text, nf_positiu, nf_count_hashtags, nf_pos_first_hashtag)


    except (KeyError):
        print "JSON format error", KeyError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
    except (ValueError):
        print "JSON format error", ValueError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
    except (TypeError):
        print "JSON format error", TypeError, index
        #print json.dumps(decoded, sort_keys=True, indent=4)
        exception = exception + 1
        

print total, index, exception

['terrorism']
7 999 0


In [92]:
file.close()